In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Argument():
    def __init__(self, batch_size=64, test_batch_size=1000, epochs=14, lr=1.0,
                gamma=0.7,no_cuda=False, save_model=False):
        
        self.batch_size = batch_size
        self.test_batch_size = test_batch_size
        self.epochs = epochs
        self.lr = lr
        self.gamma = gamma
        self.no_cuda = no_cuda
        self.save_model = save_model


In [3]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(1600, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(args, model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(args, model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\n Epoch {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100.*correct / len(test_loader.dataset)


def main():
    args = Argument()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer)
        test_acc = test(args, model, device, test_loader, epoch)
        if test_acc >= 99:
            return
        scheduler.step()

    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")


In [4]:
%%timeit -n 1 -r 5
main()


 Epoch 1 Test set: Average loss: 0.0532, Accuracy: 9850/10000 (98%)


 Epoch 2 Test set: Average loss: 0.0307, Accuracy: 9902/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0471, Accuracy: 9858/10000 (99%)


 Epoch 2 Test set: Average loss: 0.0331, Accuracy: 9903/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0383, Accuracy: 9869/10000 (99%)


 Epoch 2 Test set: Average loss: 0.0304, Accuracy: 9902/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0428, Accuracy: 9857/10000 (99%)


 Epoch 2 Test set: Average loss: 0.0308, Accuracy: 9888/10000 (99%)


 Epoch 3 Test set: Average loss: 0.0250, Accuracy: 9917/10000 (99%)


 Epoch 1 Test set: Average loss: 0.0385, Accuracy: 9866/10000 (99%)


 Epoch 2 Test set: Average loss: 0.0315, Accuracy: 9891/10000 (99%)


 Epoch 3 Test set: Average loss: 0.0243, Accuracy: 9926/10000 (99%)

30.3 s ± 5.28 s per loop (mean ± std. dev. of 5 runs, 1 loop each)
